In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/two-sigma-connect/test.csv')
data

In [ ]:
train = data.drop(data[data['target'].isna()].index.tolist())
train

### Значимость признаков feature1 и feature2

In [ ]:
from scipy.stats import pearsonr

In [ ]:
corr, _ = pearsonr(train['target'], train['feature1'])
print('Pearsons correlation: %.3f' % corr)

In [ ]:
corr, _ = pearsonr(train['target'], train['feature2'])
print('Pearsons correlation: %.3f' % corr)

In [ ]:
corr, _ = pearsonr(train['feature1'], train['feature2'])
print('Pearsons correlation: %.3f' % corr)

feature2 можно считать незначимым признаком

### events_ordinal_number - порядковый номер события

In [ ]:
# Отсортируем по user_id и time
data_sort = data.sort_values(['user_id', 'time'])
data_sort.reset_index(drop=True, inplace=True)
data_sort

In [ ]:
ev_or_num = [1]
num = 2
for i in range(1, len(data_sort)):
    if data_sort.iloc[i]['user_id'] == data_sort.iloc[i-1]['user_id']:
        ev_or_num.append(num)
        num += 1
    else:
        ev_or_num.append(1)
        num = 2

data_sort['ev_or_num'] = ev_or_num
data_sort[['ev_or_num', 'events_ordinal_number']]

### second_event_time - время второго события

In [ ]:
data_sort[data_sort['second_event_time'].isna()]

In [ ]:
# Вариант 1
def sec_ev_t(row):
    if len(data_sort[data_sort['user_id'] == row['user_id']]) <= 1:
        return None
    return data_sort[(data_sort['ev_or_num'] == 2) & (data_sort['user_id'] == row['user_id'])]['time'].values[0]
    
data_sort['sec_ev_t'] = data_sort.apply(sec_ev_t, axis=1)
data_sort[['sec_ev_t', 'second_event_time']]    

In [ ]:
# Вариант 2
sec_ev_t = []
for i in range(len(data_sort)):
    if len(data_sort[data_sort['user_id'] == data_sort.iloc[i]['user_id']]) > 1:
        sec_ev_t.append(data_sort[(data_sort['ev_or_num'] == 2) & (data_sort['user_id'] == data_sort.iloc[i]['user_id'])]['time'].values[0])
    else:
        sec_ev_t.append(None)
    
data_sort['sec_ev_t'] = sec_ev_t
data_sort[['sec_ev_t', 'second_event_time']]

### loan_ordinal_number - порядковый номер займа. 
Считается как для займов так и для заявок. Заявка всегда имеет номер займа, на который она подается. Если перед получением первого займа было 5 заявок, то все они будут иметь значение признака 1

In [ ]:
data_sort[data_sort['user_id'] == 2]

In [ ]:
# Вариант 1
def loan_or_num(row):
    if row['ev_or_num'] == 1:
        return 1
    if len(data_sort[(data_sort['user_id'] == row['user_id']) & (data_sort['ev_or_num'] <= row['ev_or_num']) & (data_sort['type'] == 'loan')]) == 0:
        return 1
    num = data_sort[(data_sort['user_id'] == row['user_id']) & (data_sort['ev_or_num'] <= row['ev_or_num']) & (data_sort['type'] == 'loan')]['type'].value_counts().values[0]
    if row['type'] == 'loanRequest':
        return num + 1
    else:
        return num
        
data_sort['loan_or_num'] = data_sort.apply(loan_or_num, axis=1)
data_sort[data_sort['user_id'] == 0][['loan_or_num', 'loan_ordinal_number']]

In [ ]:
# Вариант 2
loan_or_num = []
num = 1
for i in range(len(data_sort)):
    if i == 0:
        loan_or_num.append(1)
        if data_sort.iloc[i]['type'] == 'loan':
            num += 1
        continue
    if data_sort.iloc[i]['user_id'] == data_sort.iloc[i-1]['user_id']:
        if data_sort.iloc[i]['type'] == 'loan':
            loan_or_num.append(num)
            num += 1
        else:
            loan_or_num.append(num)
    else:
        num = 1
        loan_or_num.append(num)

data_sort['loan_or_num'] = loan_or_num
data_sort[['loan_or_num', 'loan_ordinal_number']]

In [ ]:
data_sort[data_sort['user_id'] == 0][['loan_or_num', 'loan_ordinal_number']]

### events_time_diff - разница во времени между событиями

In [ ]:
data_sort[data_sort['user_id'] == 0]

In [ ]:
data_sort[['time', 'sec_ev_t']] = data_sort[['time', 'sec_ev_t']].apply(pd.to_datetime)
data_sort.info()

In [ ]:
def ev_diff_t(row):
    if row['ev_or_num'] == 1:
        return None
    return row['time'] - data_sort[(data_sort['user_id'] == row['user_id']) & (data_sort['ev_or_num'] == row['ev_or_num'] - 1)]['time'].values[0]

data_sort['ev_diff_t'] = data_sort.apply(ev_diff_t, axis=1)

data_sort[['ev_diff_t', 'events_time_diff']]

### previous_loans_max_amount - максимальная сумма предыдущего займа

In [ ]:
data_sort[data_sort['user_id'] == 0]

In [ ]:
pr_amount = []
for i in range(len(data_sort)):
    if data_sort.iloc[i]['ev_or_num'] == 1:
        amount = None
        pr_amount.append(amount)
    else:
        pr_amount.append(amount)
    if data_sort.iloc[i]['type'] == 'loan':
        amount = data_sort.iloc[i]['amount']

data_sort['pr_amount'] = pr_amount

data_sort[['pr_amount', 'previous_loans_max_amount']]

In [ ]:
data_sort[data_sort['user_id'] == 500][['pr_amount', 'previous_loans_max_amount']]